In [1]:
import numpy as np
import pandas as pd

In [2]:
def feature_addition(csv_path):
    img_features = pd.read_csv(csv_path, index_col=0)

    img_features = img_features.assign(key=1).merge(img_features.assign(key=1), on="key", suffixes=["_A", "_B"]).drop("key", axis=1)
    img_features = img_features[img_features["img_id_A"] != img_features["img_id_B"]]
    
    return img_features

def feature_subtraction(csv_path, num_features):
    img_features = pd.read_csv(csv_path, index_col=0)
    
    img_features = img_features.assign(key=1).merge(img_features.assign(key=1), on="key", suffixes=["_A", "_B"]).drop("key", axis=1)
    img_features = img_features[img_features["img_id_A"] != img_features["img_id_B"]]
    
    
    feature_columns = [ ("f%d_%s" % (idx, label)) for label in ["A", "B"] for idx in range(1, num_features+1) ]
                                  
    for idx in range(1, num_features):
        img_features["f%d" % idx] = np.abs(img_features["f%d_A" % idx] - img_features["f%d_B" % idx]) 
     
    for label in ["A", "B"]:
        for idx in range(1, num_features+1):
            del img_features["f%d_%s" % (idx, label)]
    
    return img_features


In [3]:
# img_features = feature_addition("./HumanObserved-Dataset/HumanObserved-Features-Data/HumanObserved-Features-Data.csv")

#img_features = feature_subtraction("./HumanObserved-Dataset/HumanObserved-Features-Data/HumanObserved-Features-Data.csv", 9)
# img_features

In [4]:
def read_labeled_data(csv_paths):
    labeled_data = pd.read_csv(csv_paths[0])
    for path in csv_paths[1:]:
        labeled_data = pd.concat([labeled_data, pd.read_csv(path)])
    
    return labeled_data

In [5]:
# csv_paths = ["./HumanObserved-Dataset/HumanObserved-Features-Data/diffn_pairs.csv", "./HumanObserved-Dataset/HumanObserved-Features-Data/same_pairs.csv"]
# labeled_data = read_labeled_data(csv_paths)

In [6]:
def filter_writer_pairs(img_features, labeled_data):
    # filter from all combinations of writer pairs
    a = img_features["img_id_A"] + img_features["img_id_B"]
    b = labeled_data["img_id_A"] + labeled_data["img_id_B"]
    feature_set = img_features[a.isin(b)]
    
    feature_set = pd.merge(feature_set, labeled_data, on=["img_id_A", "img_id_B"])
    
    feature_set["writer_A"] = [elm[:4] for elm in feature_set["img_id_A"]]
    return feature_set

In [7]:
# feature_set = filter_writer_pairs(img_features, labeled_data)

In [8]:
def train_val_test_split(feature_set):
    unique = feature_set["writer_A"].unique()
    unique = np.random.permutation(unique)

    # training validation and test sets split
    tr_idx = int(0.8 * unique.shape[0])
    tr_s = unique[:tr_idx]

    training_set = feature_set.loc[feature_set["writer_A"].isin(tr_s)]


    val_idx = tr_idx + int(0.1 * unique.shape[0])
    val_s = unique[tr_idx: val_idx]
    validation_set = feature_set.loc[feature_set["writer_A"].isin(val_s)]


    test_s = unique[val_idx:]
    test_set = feature_set.loc[feature_set["writer_A"].isin(test_s)]
    
    del training_set["writer_A"]
    del validation_set["writer_A"]
    del test_set["writer_A"]
    
    return training_set, validation_set, test_set

In [9]:
# training_set, validation_set, test_set = train_val_test_split(feature_set)

In [10]:
# number of features is dynamic here
def get_features_and_labels(dataset, num_features, feature_extraction_type):
    if feature_extraction_type == "addition":
        feature_columns = [ ("f%d_%s" % (idx, label)) for label in ["A", "B"] for idx in range(1, num_features+1) ]
    elif feature_extraction_type == "subtraction":
        feature_columns = [ ("f%d" % idx) for idx in range(1, num_features+1) ]
    return np.array(dataset.loc[:, feature_columns]), np.array(dataset["target"])


In [14]:
def get_data_split(dataset_type, feature_extraction_type):

#     img_features = feature_addition("./HumanObserved-Dataset/HumanObserved-Features-Data/HumanObserved-Features-Data.csv", 9)
    
    if dataset_type == "human":
        csv_features_path = "./HumanObserved-Dataset/HumanObserved-Features-Data/HumanObserved-Features-Data.csv"
        csv_paths = ["./HumanObserved-Dataset/HumanObserved-Features-Data/diffn_pairs.csv", "./HumanObserved-Dataset/HumanObserved-Features-Data/same_pairs.csv"]
        num_features = 9
    elif dataset_type == "gsc":
        csv_features_path = "./GSC-Dataset/GSC-Features-Data/GSC-Features.csv"
        csv_paths = ["./GSC-Dataset/GSC-Features-Data/diffn_pairs.csv", "./GSC-Dataset/GSC-Features-Data/same_pairs.csv"]
        num_features = 512
        
    if feature_extraction_type == "addition":
        img_features = feature_addition(csv_features_path)
    elif feature_extraction_type == "subtraction":
        img_features = feature_subtraction(csv_features_path, num_features)

    labeled_data = read_labeled_data(csv_paths)
    
    filtered_feature_set = filter_writer_pairs(img_features, labeled_data)
    
    training_set, validation_set, test_set = train_val_test_split(filtered_feature_set)
    
    X_train, y_train = get_features_and_labels(training_set, num_features=9, feature_extraction_type=feature_extraction_type)
    X_val, y_val = get_features_and_labels(validation_set, num_features=9, feature_extraction_type=feature_extraction_type)
    X_test, y_test = get_features_and_labels(test_set, num_features=9, feature_extraction_type=feature_extraction_type)

    return X_train, y_train, X_val, y_val, X_test, y_test

# X_train, y_train, X_val, y_val, X_test, y_test = get_data_split(dataset_type="human", feature_extraction_type="addition")
# X_train, y_train, X_val, y_val, X_test, y_test = get_data_split(dataset_type="gsc", feature_extraction_type="subtraction")
X_train, y_train, X_val, y_val, X_test, y_test = get_data_split(dataset_type="human", feature_extraction_type="subtraction")

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

# i = 0
# num = X_train.shape[0] // 100

# loss = 0
# while i < X_train.shape[0]:
#     x_tr = X_train[i:i+num]
#     y_tr = y_train[i:i+num]
#     W = np.ones((x_tr.shape[1], 1))
#     prd = np.dot(x_tr, W)
#     h_x = 1 / (1 + np.exp(-prd))

# #     loss += 
#     print(np.sum(-np.log(h_x) * y_tr - np.log(1 - h_x) * (1 - y_tr)))
#     i += num

# print(loss)
# # from scipy.sparse import csr_matrix

# # feature_columns = [ ("f%d_%s" % (idx, label)) for label in ["A", "B"] for idx in range(1, 10) ]

# # X_train, y_train = csr_matrix(training_set.loc[:, feature_columns]), csr_matrix(training_set["target"])


# # W = csr_matrix(np.ones((X_train.shape[1], 1)))

# # prd = np.dot(X_train, W)

# # h_x = 1 / (2 + np.expm1(-prd))


# # # np.dot(X_train)
# # loss = np.sum(-y_train * np.log(h_x) - (1 - y) * np.log(1 - h_x))
# # print(loss)

(237100, 9) (237100,) (29127, 9) (29127,) (27596, 9) (27596,)


C:\Users\socket_var\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
